In [1]:
import gurobipy as gp
import numpy as np

In [2]:
coefficient = dict()
# coefficient['y'] = -np.r_[[1.045], [0] * 10]
coefficient['y'] = np.r_[[-1.045], [0] * 10]
coefficient['x'] = -np.r_[1.01:1.10:0.01]
# coefficient['b'] = np.r_[[1000], np.r_[[100] * 10]]
coefficient['b'] = np.r_[[1000], [100] * 10]

In [3]:
coefficient

{'y': array([-1.045,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ]),
 'x': array([-1.01, -1.02, -1.03, -1.04, -1.05, -1.06, -1.07, -1.08, -1.09,
        -1.1 ]),
 'b': array([1000,  100,  100,  100,  100,  100,  100,  100,  100,  100,  100])}

In [4]:
from typing import Any

# 继承type，使Singleton成为元类，以Singleton为metaclass的类在实例化的时候调用Singleton的__call__()函数
class Singleton(type):
    _instance = {}

    def __call__(cls, *args: Any, **kwds: Any) -> Any:
        if cls not in cls._instance:
            cls._instance[cls] = super(Singleton, cls).__call__(*args,  **kwds)
        return cls._instance[cls]

In [5]:
m = gp.Model()
x = m.addMVar(10, lb=0, ub=100, vtype=gp.GRB.CONTINUOUS, name="x")
y = m.addMVar(1, vtype=gp.GRB.INTEGER, name='y')
m.setObjective(1.045 * y - (coefficient['x'] * x).sum(), gp.GRB.MAXIMIZE)
m.addConstr(y + x.sum() - 1000 <= 0)
m.setParam('OutputFlag', 0)
m.optimize()
print(x.X)
print(y.X)
print(m.ObjVal)

Restricted license - for non-production use only - expires 2025-11-24
[  0.   0.   0.   0. 100. 100. 100. 100. 100. 100.]
[400.]
1063.0


In [6]:
from typing import Dict

class MP(metaclass=Singleton):
    def __init__(self, coefficient: Dict[str, np.ndarray]) -> None:
        self.master_problem = gp.Model("Master Problem")
        self.master_problem.setParam('OutputFlag', 0)
        self.y = self.master_problem.addMVar(1, vtype=gp.GRB.INTEGER, name="y")
        self.y = np.array(self.y.tolist())
        self.y = np.concatenate([self.y, self.master_problem.addMVar(coefficient['x'].shape, vtype=gp.GRB.INTEGER, lb=0, ub=0, name='y_pad').tolist()]).tolist()
        self.y = gp.MVar.fromlist(self.y)
        
        self.q = self.master_problem.addMVar(1, lb=-float("inf"), vtype=gp.GRB.CONTINUOUS, name="q")
        self.master_problem.setObjective((coefficient['y'] * self.y).sum() + 1 * self.q, sense=gp.GRB.MINIMIZE)

        self.rhs = coefficient["b"] - self.y
    def update_constr(self, extrem_pnt: np.ndarray = None, extrem_ray: np.ndarray = None):
        if extrem_pnt is not None and extrem_pnt.size > 0:
            # 最优性约束
            self.master_problem.addConstr((extrem_pnt * self.rhs).sum() - self.q <= 0, name='optimal_constr')
        if extrem_ray is not None and extrem_ray.size > 0:
            # 可行约束
            self.master_problem.addConstr((extrem_ray * self.rhs).sum() <= 0, name='feasible_constr')
    def solve(self):
        self.master_problem.write('master_problem.lp')
        self.master_problem.optimize()
        return self.master_problem.ObjVal
    def get_y_value(self):
        return self.y.X

In [7]:
mp = MP(coefficient)
mp.rhs
# coefficient["b"] - mp.y

<MLinExpr (11,)  >
array([ 1000.0 + -1.0 <gurobi.Var *Awaiting Model Update*>,
        100.0 + -1.0 <gurobi.Var *Awaiting Model Update*>,
        100.0 + -1.0 <gurobi.Var *Awaiting Model Update*>,
        100.0 + -1.0 <gurobi.Var *Awaiting Model Update*>,
        100.0 + -1.0 <gurobi.Var *Awaiting Model Update*>,
        100.0 + -1.0 <gurobi.Var *Awaiting Model Update*>,
        100.0 + -1.0 <gurobi.Var *Awaiting Model Update*>,
        100.0 + -1.0 <gurobi.Var *Awaiting Model Update*>,
        100.0 + -1.0 <gurobi.Var *Awaiting Model Update*>,
        100.0 + -1.0 <gurobi.Var *Awaiting Model Update*>,
        100.0 + -1.0 <gurobi.Var *Awaiting Model Update*>])

In [8]:
from typing import Dict

class SUBP(metaclass=Singleton):
    def __init__(self, coefficient: Dict[str, np.ndarray]) -> None:
        self.sub_problem = gp.Model("Sub Problem")
        self.sub_problem.setParam('OutputFlag', 0)
        self.x = self.sub_problem.addMVar(coefficient['x'].shape, vtype=gp.GRB.CONTINUOUS, name="x")
        self.sub_problem.setObjective((coefficient['x'] * self.x).sum(), gp.GRB.MINIMIZE)
    def update_y_value(self, y_value: list):
        self.y_value = y_value
        cur_constraints = self.sub_problem.getConstrs()
        self.sub_problem.remove(cur_constraints)
        self.sub_problem.update()
        self.sub_problem.addConstr(self.x.sum() - 1000 + self.y_value[0] <= 0, name='sub_problem_constr')
        for i in range(coefficient['x'].shape[0]):
            self.sub_problem.addConstr(self.x[i] <= 100)
        self.sub_problem.write('sub_problem.lp')
        
    def get_sub_pbm_res(self):
        self.sub_problem.setParam('InfUnbdInfo', 1)
        self.sub_problem.optimize()
        return self.sub_problem.Status

In [9]:
mp = MP(coefficient)
lb = mp.solve()
print(f"Current lower bound is {lb}")
y_value = mp.get_y_value()
print(f"Current y is {y_value}")
# y_value = 1500

Current lower bound is -2.045e+30
Current y is [ 1.e+30 -0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00
 -0.e+00 -0.e+00]


In [22]:
sp = SUBP(coefficient)
sp.update_y_value(y_value)
if sp.get_sub_pbm_res() == 2:
    # 子问题可行，更新原问题上界
    Pi_list = []
    for c in sp.sub_problem.getConstrs():
        Pi_list.append(c.Pi)
    print(f"Current Pi is {Pi_list}")
    Q_y = sp.sub_problem.ObjVal
    print(f'Q_y: {Q_y}')
    print(f"Current ub{Q_y + y_value * coefficient['y']}")
else:
    # 子问题不可行，更新原问题上界
    farkasdual_list = []
    for c in sp.sub_problem.getConstrs():
        farkasdual_list.append(-c.farkasdual)
    print(f"Current farkasdual is {farkasdual_list}")

Current Pi is [-1.05, 0.0, 0.0, 0.0, 0.0, 0.0, -0.010000000000000009, -0.020000000000000018, -0.030000000000000027, -0.040000000000000036, -0.050000000000000044]
Q_y: -592.5
Current ub[-1062.75  -592.5   -592.5   -592.5   -592.5   -592.5   -592.5   -592.5
  -592.5   -592.5   -592.5 ]


In [11]:
sp.sub_problem

<gurobi.Model Continuous instance Sub Problem: 11 constrs, 10 vars, Parameter changes: InfUnbdInfo=1, OutputFlag=0>

In [11]:
mp = MP(coefficient)
mp.update_constr(extrem_ray=np.array(farkasdual_list))
lb = mp.solve()
print(f"Current lower bound is {lb}")
y_value = mp.get_y_value()
print(f"Current y is {y_value}")

Current lower bound is -1e+30
Current y is [1000.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.]


In [21]:
mp = MP(coefficient)
mp.update_constr(extrem_pnt=np.array(Pi_list))
lb = mp.solve()
print(f"Current lower bound is {lb}")
y_value = mp.get_y_value()
print(f"Current y is {y_value}")

Current lower bound is -1063.25
Current y is [450.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
